
# 🛠️ บทสรุป: การสร้างระบบหลายเอเจนต์สำหรับแคมเปญการเข้าถึงลูกค้าโดยใช้เครื่องมือ

## 📖 บทนำ
ในบทเรียนนี้ เราจะสร้างระบบหลายเอเจนต์ (Multi-Agent System) เพื่อใช้ในการสร้างแคมเปญการเข้าถึงลูกค้า โดยใช้ crewAI และเครื่องมือต่างๆ ซึ่งเป็นกรณีการใช้งานที่สำคัญในด้านการขาย

## 🚀 การตั้งค่าเริ่มต้น
1. **นำเข้าโมดูลที่จำเป็น**
```python
from crewai import Agent, Task, Crew
```
นำเข้าโมดูลหลัก: Agent, Task, และ Crew

2. **กำหนดโมเดล LLM**
ใช้โมเดล GPT-4 เนื่องจากเราจะจัดการกับข้อมูลที่มากขึ้น
```python
import os
os.environ["OPENAI_MODEL_NAME"] = "gpt-4"
```

3. **ตั้งค่า Serper API**
ใช้ Serper API เพื่อให้เอเจนต์สามารถค้นหาข้อมูลจาก Google คุณสามารถสมัครใช้งานและรับคีย์ API ได้ฟรี

## 👥 การสร้างเอเจนต์
1. **สร้างเอเจนต์ตัวแทนฝ่ายขาย (Sales Representative Agent)**
- **บทบาท:** ตัวแทนฝ่ายขาย
- **เป้าหมาย:** ระบุลูกค้าที่มีมูลค่าสูงที่ตรงกับโปรไฟล์ลูกค้าเป้าหมาย (ICP)
- **เรื่องราวพื้นหลัง:** ทำงานที่ crewAI
- **ข้อจำกัด:** ไม่อนุญาตให้มอบหมายงานหรือถามคำถาม
```python
sales_rep_agent = Agent(
    role="ตัวแทนฝ่ายขาย",
    goal="ระบุลูกค้าที่มีมูลค่าสูงที่ตรงกับโปรไฟล์ลูกค้าเป้าหมาย",
    backstory="คุณทำงานที่ crewAI และมุ่งเน้นการหาลูกค้าใหม่",
    allow_delegate=False
)
```

2. **สร้างเอเจนต์หัวหน้าฝ่ายขาย (Lead Sales Representative Agent)**
- **บทบาท:** หัวหน้าฝ่ายขาย
- **เป้าหมาย:** สร้างการสื่อสารที่ปรับแต่งเฉพาะสำหรับลูกค้า
- **เรื่องราวพื้นหลัง:** ทำงานร่วมกับทีมเพื่อเข้าถึงลูกค้าเป้าหมาย
```python
lead_sales_rep_agent = Agent(
    role="หัวหน้าฝ่ายขาย",
    goal="สร้างการสื่อสารที่ปรับแต่งเฉพาะสำหรับลูกค้า",
    backstory="คุณเป็นหัวหน้าฝ่ายขายที่ crewAI และมุ่งเน้นการสร้างความสัมพันธ์กับลูกค้า",
    allow_delegate=True
)
```

## 🛠️ การใช้เครื่องมือ
1. **นำเข้าเครื่องมือจาก crewAI**
```python
from crewai.tools import DirectoryReadTool, FileReadTool, SerpAPITool
```

2. **สร้างและกำหนดค่าเครื่องมือ**
- **DirectoryReadTool:** ให้เอเจนต์อ่านไฟล์ในไดเรกทอรีที่ระบุ
```python
directory_read_tool = DirectoryReadTool(directory="instructions")
```

- **FileReadTool:** ให้เอเจนต์อ่านไฟล์เฉพาะ
- **SerpAPITool:** ให้เอเจนต์ค้นหาข้อมูลจากอินเทอร์เน็ต
```python
file_read_tool = FileReadTool()
search_tool = SerpAPITool()
```
> หมายเหตุ: ไดเรกทอรี `instructions` มีไฟล์ Markdown ที่มีคำแนะนำในการจัดการกับบริษัทขนาดต่างๆ

## 🛠️ การสร้างเครื่องมือแบบกำหนดเอง
- **สร้างเครื่องมือวิเคราะห์ความรู้สึก (Sentiment Analysis Tool)**  
  วิธีการสร้าง:
  - นำเข้า `BaseTool` จาก crewAI
  - สร้างคลาสใหม่ที่สืบทอดจาก `BaseTool`
  - กำหนดชื่อและคำอธิบายของเครื่องมือ
  - เขียนฟังก์ชัน `_run` ที่ดำเนินการตามต้องการ

```python
from crewai.tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name = "sentiment_analysis_tool"
    description = "วิเคราะห์ความรู้สึกของข้อความที่กำหนด"

    def _run(self, text):
        # ตัวอย่างการคืนค่าความรู้สึกเป็น 'positive' เสมอ
        return "positive"

sentiment_analysis_tool = SentimentAnalysisTool()
```

## 📋 การสร้างงาน (Tasks)
1. **งานโปรไฟล์ลูกค้า (Lead Profiling Task)**
- **รายละเอียด:** วิเคราะห์ลูกค้าเพื่อทำความเข้าใจเกี่ยวกับอุตสาหกรรม ขนาด และข้อมูลที่เกี่ยวข้อง
- **เอเจนต์:** `sales_rep_agent`
- **เครื่องมือที่ใช้:** `directory_read_tool`, `file_read_tool`, `search_tool`
```python
lead_profiling_task = Task(
    description="วิเคราะห์ลูกค้า {lead_name} เพื่อทำความเข้าใจอุตสาหกรรมและขนาดของพวกเขา",
    expected_output="โปรไฟล์ลูกค้าที่ละเอียด",
    agent=sales_rep_agent,
    tools=[directory_read_tool, file_read_tool, search_tool]
)
```

2. **งานสร้างการสื่อสารที่ปรับแต่ง (Personalized Outreach Task)**
- **รายละเอียด:** สร้างข้อความที่ปรับแต่งเฉพาะสำหรับลูกค้า
- **เอเจนต์:** `lead_sales_rep_agent`
- **เครื่องมือที่ใช้:** `sentiment_analysis_tool`, `search_tool`
```python
personalized_outreach_task = Task(
    description="สร้างข้อความที่ปรับแต่งสำหรับลูกค้า {lead_name}",
    expected_output="อีเมลหรือข้อความที่ปรับแต่งเฉพาะ",
    agent=lead_sales_rep_agent,
    tools=[sentiment_analysis_tool, search_tool]
)
```

## 🤖 การสร้าง Crew และเริ่มการทำงาน
1. **สร้าง Crew**
```python
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
    use_memory=True
)
```

2. **เริ่มการทำงานของ Crew**  
กำหนดค่าที่จะใช้แทนที่ในการทำงาน:
```python
inputs = {
    "lead_name": "DeepLearning.AI",
    "industry": "แพลตฟอร์มการเรียนรู้ออนไลน์",
    "key_decision_maker": "Andrew",
    "position": "CEO",
    "milestone": "การเปิดตัวผลิตภัณฑ์ใหม่"
}

crew.kickoff(inputs)
```

## 📊 การทำงานของเอเจนต์
1. **เอเจนต์ตัวแทนฝ่ายขาย (Sales Representative Agent)**  
   - ดำเนินการวิเคราะห์ลูกค้า: ใช้เครื่องมือค้นหาและอ่านไฟล์เพื่อรวบรวมข้อมูลเกี่ยวกับ DeepLearning.AI  
   - จัดการข้อผิดพลาดอย่างสง่างาม: หากเกิดข้อผิดพลาดในการใช้เครื่องมือ เอเจนต์จะสามารถจัดการและดำเนินการต่อได้

2. **เอเจนต์หัวหน้าฝ่ายขาย (Lead Sales Representative Agent)**  
   - สร้างข้อความที่ปรับแต่งเฉพาะ: ใช้ผลลัพธ์จากเอเจนต์ก่อนหน้าและเครื่องมือวิเคราะห์ความรู้สึก  
   - สร้างอีเมลหรือข้อความที่ปรับแต่งเพื่อเข้าถึง Andrew ที่ DeepLearning.AI

3. **การใช้แคชและการจัดการข้อผิดพลาด**
   - **การใช้แคช:** ช่วยลดการเรียกใช้งาน API ซ้ำๆ และเพิ่มประสิทธิภาพ  
   - **การจัดการข้อผิดพลาด:** crewAI มีการจัดการข้อผิดพลาดในตัว ทำให้ระบบไม่หยุดทำงานเมื่อเกิดข้อผิดพลาด

## 📬 ผลลัพธ์
ได้รับอีเมลหรือข้อความที่ปรับแต่งเฉพาะสำหรับลูกค้า ซึ่งสามารถใช้ในการเข้าถึงและสร้างความสัมพันธ์กับพวกเขา

## 📝 บทสรุป
- **การขายเป็นกรณีการใช้งานที่สำคัญสำหรับระบบหลายเอเจนต์:** การรายงาน การวิจัย และการเข้าถึงลูกค้าสามารถถูกทำให้อัตโนมัติด้วยเอเจนต์
- **เครื่องมือเป็นส่วนสำคัญในการทำให้เอเจนต์มีประสิทธิภาพ:** การใช้เครื่องมือที่เหมาะสมช่วยให้เอเจนต์สามารถดำเนินการที่ซับซ้อนได้
- **การจัดการข้อผิดพลาดและการใช้แคช:** ช่วยให้ระบบทำงานได้อย่างราบรื่นและมีประสิทธิภาพมากขึ้น

## 💡 ข้อเสนอแนะ
- ลองปรับแต่งเอเจนต์และเครื่องมือเพื่อให้ตรงกับความต้องการเฉพาะของคุณ
- พิจารณาเพิ่มเอเจนต์หรือเครื่องมือเพิ่มเติมเพื่อปรับปรุงระบบของคุณ
- สำรวจการใช้ crewAI ในกรณีการใช้งานอื่นๆ ที่เกี่ยวข้องกับธุรกิจของคุณ


# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our sales team.

Action: Read a file's content
Action Input: {"file_path": "company_profiles/DeepLearningAI.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: 'company_profiles/DeepLearningAI.txt'.
 Tool Read a file's content accepts 

Thought: I now need to compile all the information gathered into a comprehensive report on DeepLearningAI for our sales team.

Final Answer: 
DeepLearningAI is an online learning platform founded by Andrew Ng, a pioneer in the field of deep learning and artificial intelligence. The company offers AI courses and specializations to over 7 million learners worldwide. Andrew Ng serves as the Founder and CEO of DeepLearningAI and has a strong background in AI research and education.

Recent milestones for DeepLearningAI include the creation of quality courses that prioritize learner success and the continuous delivery of AI news and insights through their platform, "The Batch." Additionally, Andrew Ng's appointment to Amazon's Board of Directors further solidifies his influence and expertise in the AI industry.

In terms of potential needs that align with our offerings, DeepLearningAI may benefit from a personalized engagement strategy that focuses on enhancing their course offerings with c

Thought: I need to incorporate information about DeepLearningAI's recent product launch into my outreach campaign to Andrew Ng.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI recent product launch 2024"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Search the internet
Action Input: {"search_query": "DeepLearningAI latest product launch details"} 


Search results: Title: DeepLearning.AI: Start or Advance Your Career in AI
Link: https://www.deeplearning.ai/
Snippet: Join over 7 million people learning how to use and build AI. AI Courses and Specializations. Build a foundation of machine learning and AI skills.
---
Title: The Batch | DeepLearning.AI | AI News & Insights
Link: https://www.deeplearning.ai/the-batch/
Snippet: The Batch AI News and Insights: After a recent price reduction by OpenAI, GPT-4o tokens now cost $4 per million tokens (using a blended rate that assumes 80% ...
---
Titl

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating DeepLearningAI's Recent Product Launch with Our Cutting-Edge Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to take a moment to congratulate you and the entire team at DeepLearningAI on the recent launch of your latest product. The innovative strides you are taking in the field of AI education are truly inspiring.

As a pioneer in the field of deep learning and artificial intelligence, your commitment to providing high-quality courses and specializations to over 7 million learners worldwide is commendable. The continuous delivery of AI news and insights through "The Batch" showcases your dedication to keeping the community informed and engaged.

I recently came across the news of your latest product launch, which has garnered significant attention in the industry. The advancements you are making in AI education are setting new standards and reshaping the future of learning. We believe that our cutting-edge tools and technologies can further enhance your course offerings, providing your users with an unparalleled learning experience.

At CrewAI, we specialize in personalized engagement strategies that optimize the learning journey and drive learner success. By leveraging our solutions, you can stay ahead of the competition and continue to deliver exceptional value to your users. We are excited about the opportunity to collaborate with DeepLearningAI and support your mission of advancing AI education.

I would love to discuss how our solutions can complement your recent product launch and help you achieve your business goals. Please let me know a convenient time for a call or meeting to explore this further.

Thank you for your time, Andrew. I look forward to the possibility of working together and contributing to the continued success of DeepLearningAI.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

[Contact Information]